In [1]:
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Frank") \
      .getOrCreate() 

# Einlesen von Dateien in Spark, Datei Typ einfach Anpassen
frank=spark.read.text("frankenstein.txt")
frank.show(truncate=False)
# Schema anzeigen lassen


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/19 13:57:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+-------------------------------------------------------------------------+
|value                                                                    |
+-------------------------------------------------------------------------+
|                                                                         |
|Project Gutenberg's Frankenstein, by Mary Wollstonecraft (Godwin) Shelley|
|                                                                         |
|This eBook is for the use of anyone anywhere at no cost and with         |
|almost no restrictions whatsoever.  You may copy it, give it away or     |
|re-use it under the terms of the Project Gutenberg License included      |
|with this eBook or online at www.gutenberg.net                           |
|                                                                         |
|                    

Nur Wörter anzeigen lassen, welche Wörter sind 

In [7]:

import pyspark.sql.functions as F
lines=frank.select(F.split(frank.value,"[^a-zA-Z]").alias("Zeile"))
lines.show(truncate=False)


+--------------------------------------------------------------------------------------+
|Zeile                                                                                 |
+--------------------------------------------------------------------------------------+
|[]                                                                                    |
|[Project, Gutenberg, s, Frankenstein, , by, Mary, Wollstonecraft, , Godwin, , Shelley]|
|[]                                                                                    |
|[This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with]       |
|[almost, no, restrictions, whatsoever, , , You, may, copy, it, , give, it, away, or]  |
|[re, use, it, under, the, terms, of, the, Project, Gutenberg, License, included]      |
|[with, this, eBook, or, online, at, www, gutenberg, net]                              |
|[]                                                                                    |
|[]                  

Jetz leere Zeilen entfernen

In [9]:
rmempty=lines.select(F.array_remove(F.col("Zeile"),"").alias("zeile"))
rmempty.show(truncate=False)


+--------------------------------------------------------------------------------+
|zeile                                                                           |
+--------------------------------------------------------------------------------+
|[]                                                                              |
|[Project, Gutenberg, s, Frankenstein, by, Mary, Wollstonecraft, Godwin, Shelley]|
|[]                                                                              |
|[This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with] |
|[almost, no, restrictions, whatsoever, You, may, copy, it, give, it, away, or]  |
|[re, use, it, under, the, terms, of, the, Project, Gutenberg, License, included]|
|[with, this, eBook, or, online, at, www, gutenberg, net]                        |
|[]                                                                              |
|[]                                                                              |
|[Ti

Wörter in den einzelenen Zeilen zählen und darstellen

In [28]:
count_words=rmempty.withColumn("Anzahl",F.size(F.col("zeile")))
count_words.show()
count_words.count()


+--------------------+------+
|               zeile|Anzahl|
+--------------------+------+
|                  []|     0|
|[Project, Gutenbe...|     9|
|                  []|     0|
|[This, eBook, is,...|    14|
|[almost, no, rest...|    12|
|[re, use, it, und...|    12|
|[with, this, eBoo...|     9|
|                  []|     0|
|                  []|     0|
|[Title, Frankenst...|     2|
|[or, The, Modern,...|     4|
|                  []|     0|
|[Author, Mary, Wo...|     5|
|                  []|     0|
|[Release, Date, J...|     4|
|[Last, updated, J...|     3|
|                  []|     0|
| [Language, English]|     2|
|                  []|     0|
|[Character, set, ...|     4|
+--------------------+------+
only showing top 20 rows

DataFrame[zeile: array<string>, Anzahl: int]


Leere Arrays entfernen

In [ ]:
#leere Arrays entfernen 

clean_word=count_words.where(F.col("Anzahl")!=0)
clean_word.show()


+--------------------+------+
|               zeile|Anzahl|
+--------------------+------+
|[Project, Gutenbe...|     9|
|[This, eBook, is,...|    14|
|[almost, no, rest...|    12|
|[re, use, it, und...|    12|
|[with, this, eBoo...|     9|
|[Title, Frankenst...|     2|
|[or, The, Modern,...|     4|
|[Author, Mary, Wo...|     5|
|[Release, Date, J...|     4|
|[Last, updated, J...|     3|
| [Language, English]|     2|
|[Character, set, ...|     4|
|[START, OF, THIS,...|     7|
|[Produced, by, Ju...|     8|
|[and, David, Melt...|     8|
|[Further, correct...|     6|
|      [Frankenstein]|     1|
|[or, the, Modern,...|     4|
|                [by]|     1|
|[Mary, Wollstonec...|     4|
+--------------------+------+
only showing top 20 rows



Anzeigen welche Zeile die meisten Einträge hat

In [19]:
#Anzeigen welche Zeile die meisten Einträge hat
sort=clean_word.orderBy(F.col("Anzahl").desc())
sort.show()


+--------------------+------+
|               zeile|Anzahl|
+--------------------+------+
|[seas, I, have, f...|    18|
|[As, I, stood, at...|    18|
|[and, as, far, as...|    18|
|[but, if, it, be,...|    18|
|[and, was, led, t...|    18|
|[me, the, rest, I...|    18|
|[will, pass, befo...|    17|
|[absence, of, the...|    17|
|[and, beloved, as...|    17|
|[country, but, it...|    17|
|[so, but, then, h...|    17|
|[in, her, innocen...|    17|
|[cousin, My, fath...|    17|
|[is, all, that, I...|    17|
|[out, of, the, ro...|    17|
|[for, port, It, w...|    17|
|[For, a, long, ti...|    17|
|[can, do, and, I,...|    17|
|[I, swear, to, yo...|    17|
|[but, I, sailed, ...|    17|
+--------------------+------+
only showing top 20 rows



In [29]:
#Exploden der Liste
explode_words=sort.select(F.explode(F.col("zeile")).alias("Word"))
explode_words.show()

+------+
|  Word|
+------+
|    As|
|     I|
| stood|
|    at|
|   the|
|  door|
|    on|
|     a|
|sudden|
|     I|
|beheld|
|     a|
|stream|
|    of|
|  fire|
| issue|
|  from|
|    an|
|   and|
|   was|
+------+
only showing top 20 rows



78444

In [23]:
# Wörter lower_case machen
lower_words=explode_words.select(F.lower(F.col("Word")).alias("lower_words"))
lower_words.show()

+-----------+
|lower_words|
+-----------+
|         as|
|          i|
|      stood|
|         at|
|        the|
|       door|
|         on|
|          a|
|     sudden|
|          i|
|     beheld|
|          a|
|     stream|
|         of|
|       fire|
|      issue|
|       from|
|         an|
|        and|
|        was|
+-----------+
only showing top 20 rows



In [27]:
count_word=(
    lower_words.select(F.col("lower_words"))
    .count()
)
print(count_word)

78444


In [33]:
#Entfernen von Wörtern die kleiner als 2 Zeichen sind ausßer a und i 
count_uniq=lower_words.select(F.regexp_extract(F.col("lower_words"),"[a-z]{2,}|a|i",0).alias("uniq"))
count_uniq.show()

+------+
|  uniq|
+------+
|    as|
|     i|
| stood|
|    at|
|   the|
|  door|
|    on|
|     a|
|sudden|
|     i|
|beheld|
|     a|
|stream|
|    of|
|  fire|
| issue|
|  from|
|    an|
|   and|
|   was|
+------+
only showing top 20 rows



In [36]:
#leere zeilen wieder entfernen
counted_uniq=count_uniq.count()
print(counted_uniq)
print(count_uniq)
rmempty=count_uniq.filter(F.col("uniq")!="")
rmempty.show()

counted_rm=rmempty.count()
print(counted_rm)

78444
DataFrame[uniq: string]
+------+
|  uniq|
+------+
|    as|
|     i|
| stood|
|    at|
|   the|
|  door|
|    on|
|     a|
|sudden|
|     i|
|beheld|
|     a|
|stream|
|    of|
|  fire|
| issue|
|  from|
|    an|
|   and|
|   was|
+------+
only showing top 20 rows

78236


In [58]:
#Gleiche Wörter gruppieren und diese zählen
grouped=rmempty.groupBy(F.col("uniq"))
counted_group=grouped.count().sort(F.desc("count"))
counted_group.show()

+-----+-----+
| uniq|count|
+-----+-----+
|  the| 4371|
|  and| 3046|
|    i| 2850|
|   of| 2760|
|   to| 2174|
|   my| 1776|
|    a| 1449|
|   in| 1186|
| that| 1033|
|  was| 1022|
|   me|  868|
| with|  714|
|  but|  692|
|  had|  686|
|  you|  644|
|   he|  611|
|which|  565|
|   it|  562|
|   as|  539|
|  his|  535|
+-----+-----+
only showing top 20 rows



In [67]:
#Anzahl der Wörter welche eine x länge haben
counted_uniq_words=counted_group.select(F.length(F.col("uniq")).alias("Anzahl_Wörter")).groupBy("Anzahl_Wörter").count().orderBy(F.col("count").desc()).show()

+-------------+-----+
|Anzahl_Wörter|count|
+-------------+-----+
|            7| 1201|
|            6| 1113|
|            8| 1070|
|            9|  862|
|            5|  844|
|            4|  632|
|           10|  594|
|           11|  385|
|           12|  186|
|            3|  185|
|           13|   84|
|            2|   46|
|           14|   34|
|           15|   12|
|           16|    5|
|            1|    2|
|           18|    1|
+-------------+-----+



In [45]:
counted_group.coalesce(1).write.option("header","true").csv("counted_words.csv")

In [46]:
from pymongo import MongoClient
import csv

# specify the MongoDB server host and port
host = "localhost"
port = 27017

# create a MongoClient instance
client = MongoClient(host, port)

# specify the database and collection names
db_name = "Wörter"
collection_name = "zählen"

# get a reference to the database and collection
db = client[db_name]
collection = db[collection_name]

# open the CSV file and read the data
with open("counted_words.csv/counted_words.csv") as csv_file:
    reader = csv.DictReader(csv_file)
    data = list(reader)

# insert the data into the collection
result = collection.insert_many(data)